In [2]:
import math 
import copy
import numpy as np
import pandapower as pp
from itertools import combinations
import pandapower.networks as pn
import time

In [3]:
class shapley_calc:
    def __init__(self, net):
        self.net = net
        self.load_num = len(self.net.load)
        self.sgen_num = len(self.net.sgen)
        self.total_player_num = self.load_num+self.sgen_num

    def coalitional_value(self, coalition): # TRUE FALSE COALITION
        self.copy_net = copy.deepcopy(self.net)
        
        # SEPARATE BOOL INDEX
        self.load_bool_idx = coalition[:self.load_num]
        self.sgen_bool_idx = coalition[self.load_num:self.total_player_num]

        self.update_load_list = np.zeros(self.load_num)
        for i, value in enumerate(self.load_bool_idx):
            if value == True:
                self.update_load_list[i] = self.copy_net.load.p_mw[i]
            elif value == False:
                self.update_load_list[i] = 0

        self.update_sgen_list = np.zeros(self.sgen_num)
        for i, value in enumerate(self.sgen_bool_idx):
            if value == True:
                self.update_sgen_list[i] = self.copy_net.sgen.p_mw[i]
            elif value == False:
                
                self.update_sgen_list[i] = 0

        # UPDATE DATA
        self.copy_net.load.p_mw = self.update_load_list
        self.copy_net.sgen.p_mw = self.update_sgen_list

        pp.runpp(self.copy_net)

        return sum(self.copy_net.res_line.pl_mw)

    def component_shapley(self, player):
        self.shapley = 0
        for coalition_number in range(self.total_player_num):
            #print("coal_number : ", coalition_number)
            
            self.numbering_index = list(range(self.total_player_num))
            self.numbering_index.remove(player)

            self.without_i_coalition_list = list(combinations(self.numbering_index, coalition_number))

            for players in list(self.without_i_coalition_list):
                self.bool_list_w_player=np.zeros(self.total_player_num)
                self.bool_list_wo_player=np.zeros(self.total_player_num)
                
                for indexing_number in range(self.total_player_num):
                    if indexing_number in players:
                        self.bool_list_w_player[indexing_number] = True
                        self.bool_list_wo_player[indexing_number] = True
                    else:
                        self.bool_list_w_player[indexing_number] = False
                        self.bool_list_wo_player[indexing_number] = False
                    
                    self.bool_list_w_player[player] = True
                    self.bool_list_wo_player[player] = False

                '''print('coalition_number_wo_player : ',coalition_number)
                print('player : ', player)
                print('bool_w_player : ', self.bool_list_w_player)
                print('bool_wo_player : ', self.bool_list_wo_player)'''

                self.weight = math.factorial(coalition_number)*math.factorial(self.total_player_num-coalition_number-1)/math.factorial(self.total_player_num)
                self.marginal_value = self.coalitional_value(self.bool_list_w_player)-self.coalitional_value(self.bool_list_wo_player)
                self.shapley += self.weight * self.marginal_value
        
        return self.shapley
    
    def shapley_list(self):
        self.shapley_list = np.zeros(self.total_player_num)
        for player in range(self.total_player_num):
            self.shapley_list[player] = self.component_shapley(player=player)
        
        return self.shapley_list

In [4]:
net = pn.case33bw()
pp.runpp(net)
total_loss = sum(net.res_line.pl_mw)

start = time.time()
sv = shapley_calc(net)
loss_shapley = sv.shapley_list() # **
end = time.time()

print("Total_loss : ", total_loss)
print('calculation_time :', f"{end - start:.5f}")
print("shapley : ", loss_shapley)
print("sum_shapley : ", sum(loss_shapley))

net.load.p_mw = 0
pp.runpp(net)
print("zero_load_loss : ", sum(net.res_line.pl_mw))

print("sum zero load & shapley :", sum(loss_shapley)+sum(net.res_line.pl_mw))

KeyboardInterrupt: 